# Boolean Search in Documents

## Objective
Expand the simple term search functionality to include Boolean search capabilities. This will allow users to perform more complex queries by combining multiple search terms using Boolean operators.

## Problem Description
You must enhance the existing search engine from the previous exercise to support Boolean operators: AND, OR, and NOT. This will enable the retrieval of documents based on the logical relationships between multiple terms.

## Requirements

### Step 1: Update Data Preparation
Ensure that the documents are still loaded and preprocessed from the previous task. The data should be clean and ready for advanced querying.

### Step 2: Create an Inverted Index

Create an inverted index from the documents. This index maps each word to the set of document IDs in which that word appears. This facilitates word lookup in the search process.

### Step 3: Implementing Boolean Search
- **Enhance Input Query**: Modify the function to accept complex queries that can include the Boolean operators AND, OR, and NOT.
- **Implement Boolean Logic**:
  - **AND**: The document must contain all the terms. For example, `python AND programming` should return documents containing both "python" and "programming".
  - **OR**: The document can contain any of the terms. For example, `python OR programming` should return documents containing either "python", "programming", or both.
  - **NOT**: The document must not contain the term following NOT. For example, `python NOT snake` should return documents that contain "python" but not "snake".

### Step 4: Query Processing
- **Parse the Query**: Implement a function to parse the input query to identify the terms and operators.
- **Search Documents**: Based on the parsed query, implement the logic to retrieve and rank the documents according to the Boolean expressions.
- **Handling Case Sensitivity and Partial Matches**: Optionally, you can handle cases and partial matches to refine the search results.

In [13]:
import os
import pandas as pd
from nltk.tokenize import word_tokenize

def construir_indice_invertido():
    indice = {}
    documentos = {}  # Mantener un registro de los documentos y su contenido para referencia posterior
    
    # Obtener la lista de documentos en el directorio
    documentos_path = "Downloads/descargas"
    libros = os.listdir(documentos_path)
    
    # Iterar sobre cada documento para construir el índice invertido
    for libro in libros:
        with open(os.path.join(documentos_path, libro), 'r', encoding='utf-8') as f:
            contenido = f.read().lower()  # Convertir el contenido a minúsculas
            palabras = word_tokenize(contenido)  # Tokenizar el contenido en palabras usando NLTK
            
            # Registrar el contenido del documento para referencia posterior
            documentos[libro] = contenido
            
            for palabra in set(palabras):  # Usar un conjunto para evitar duplicados
                if palabra not in indice:
                    indice[palabra] = set()  # Usar un conjunto para almacenar los libros donde aparece la palabra
                indice[palabra].add(libro)  # Agregar el libro a la lista de libros donde aparece la palabra
    
    # Crear un DataFrame con el índice invertido
    df = pd.DataFrame(index=indice.keys(), columns=libros)
    df.fillna(False, inplace=True)  # Rellenar todos los valores con False
    
    # Marcar las celdas como True donde la palabra aparece en un libro
    for palabra, libros in indice.items():
        for libro in libros:
            df.at[palabra, libro] = True
    
    return df, documentos

indice_invertido_df, documentos = construir_indice_invertido()


### Step 5: Displaying Results
- **Output the Results**: Display the documents that match the query criteria. Include functionalities to handle queries that result in no matching documents.

In [ ]:
def buscar_palabra_en_indice(query):
    # Convertir la consulta a minúsculas para que coincida con el formato en el DataFrame
    query = query.lower()
    
    # Dividir la consulta en términos y operadores
    terminos = query.split()
    operadores = set(["and", "or", "not"])
    
    # Verificar si la consulta contiene operadores booleanos
    if any(op in operadores for op in terminos):
        # Inicializar un conjunto de documentos que coinciden con la consulta
        resultados = set(documentos.keys())  # Empezamos con todos los documentos como posibles resultados
        
        # Iterar sobre los términos y operadores para evaluar la consulta booleana
        i = 0
        while i < len(terminos):
            term = terminos[i]
            if term == "not":
                if i + 1 < len(terminos):
                    siguiente_term = terminos[i + 1]
                    if siguiente_term in indice_invertido_df.index:
                        # Si se encuentra un término después de "not", excluimos los documentos que lo contienen
                        resultados.difference_update(indice_invertido_df.loc[siguiente_term].index)
                    i += 1
            elif term == "and":
                i += 1
                if i < len(terminos):
                    siguiente_term = terminos[i]
                    if siguiente_term in indice_invertido_df.index:
                        # Si se encuentra un término después de "and", intersectamos los documentos que lo contienen
                        resultados.intersection_update(indice_invertido_df.loc[siguiente_term].index)
            elif term == "or":
                i += 1
                if i < len(terminos):
                    siguiente_term = terminos[i]
                    if siguiente_term in indice_invertido_df.index:
                        # Si se encuentra un término después de "or", unimos los documentos que lo contienen
                        resultados.update(indice_invertido_df.loc[siguiente_term].index)
            else:
                # Si el término no es un operador, no hacemos nada
                pass
            
            i += 1
        
        # Filtrar documentos que realmente coinciden con la consulta
        resultados_filtrados = [documento for documento in resultados if all(term in documentos[documento].lower() for term in terminos)]
        
        if resultados_filtrados:
            resultados_df = pd.DataFrame(index=resultados_filtrados)
            print("Documentos que coinciden con la consulta '{}':".format(query))
            display(resultados_df)
        else:
            print("No se encontraron documentos que coincidan con la consulta '{}'.\n".format(query))
        
    else:
        # Si la consulta no contiene operadores booleanos, simplemente buscamos la palabra en el índice invertido
        if query in indice_invertido_df.index:
            resultados = indice_invertido_df.loc[query]
            resultados = resultados[resultados > 0]  # Filtrar los libros donde la palabra aparece al menos una vez
            if not resultados.empty:
                resultados_df = resultados.to_frame(name='Número de ocurrencias')
                print("Resultados para la palabra '{}':".format(query))
                display(resultados_df)
            else:
                print("La palabra '{}' aparece en los libros, pero el número de ocurrencias es cero.".format(query))
        else:
            print("La palabra '{}' no se encuentra en ningún libro.".format(query))

# Interfaz de búsqueda
while True:
    consulta = input("Ingrese una consulta para buscar en el índice invertido (o 'salir' para terminar): ")
    if consulta.lower() == 'salir':
        break
    buscar_palabra_en_indice(consulta)


Ingrese una consulta para buscar en el índice invertido (o 'salir' para terminar):  gut


Resultados para la palabra 'gut':


,Número de ocurrencias
pg16.txt,True
pg1727.txt,True
pg2160.txt,True
pg2600.txt,True
pg29728.txt,True
pg37106.txt,True
pg41070.txt,True
pg4300.txt,True
pg514.txt,True
pg6761.txt,True


Ingrese una consulta para buscar en el índice invertido (o 'salir' para terminar):  gut AND romeoç


No se encontraron documentos que coincidan con la consulta 'gut and romeoç'.



Ingrese una consulta para buscar en el índice invertido (o 'salir' para terminar):  gut AND romeo


Documentos que coinciden con la consulta 'gut and romeo':


""
pg2814.txt
pg5197.txt
pg76.txt
pg37106.txt
pg8800.txt
pg52882.txt
pg100.txt
pg4300.txt
pg514.txt
pg20228.txt


## Evaluation Criteria
- **Correctness**: The Boolean search implementation should correctly interpret and process the queries according to the Boolean logic.
- **Efficiency**: Consider the efficiency of your search process, especially as the complexity of queries increases.
- **User Experience**: Ensure that the interface for inputting queries and viewing results is user-friendly.

## Additional Challenges (Optional)
- **Nested Boolean Queries**: Allow for nested queries using parentheses, such as `(python OR java) AND programming`.
- **Phrase Searching**: Implement the ability to search for exact phrases enclosed in quotes.
- **Proximity Searching**: Extend the search to find terms that are within a specific distance from one another.

This exercise will deepen your understanding of how search engines process and respond to complex user queries. By incorporating Boolean search, you not only enhance the functionality of your search engine but also mimic more closely how real-world information retrieval systems operate.